# **ライブラリインポート**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

device と seed設定

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)
torch.manual_seed(1)

device cuda


ハイパーパラメーター

In [ ]:
batch_size = 100
num_classes = 100
epochs = 150

# **データセット**

In [ ]:
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

x_train = np.moveaxis(x_train, [3, 1, 2], [1, 2, 3]).astype('float32')
x_test = np.moveaxis(x_test, [3, 1, 2], [1, 2, 3]).astype('float32')

x_train /= 255
x_test /= 255

y_train = y_train.reshape(-1).astype('long')
y_test = y_test.reshape(-1).astype('long')


169017344/169001437 [==============================] - 3s 0us/step


numpyにセット

In [ ]:
ds_train = data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
ds_test  = data.TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))

データローダー作成

In [ ]:
dataloader_train = data.DataLoader(dataset=ds_train, batch_size=batch_size, shuffle=True)

dataloader_test = data.DataLoader(dataset=ds_test, batch_size=batch_size, shuffle=False)

# **モデル作成**

In [ ]:
class Cifar100Model(nn.Module):
    def __init__(self):
        super(Cifar100Model, self).__init__()
        self.conv11 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv12 = nn.Conv2d(32, 32, 3, padding=1)

        self.conv21 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv22 = nn.Conv2d(64, 64, 3, padding=1)

        self.fc1 = nn.Linear(64 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.25)
        self.dropout3 = nn.Dropout2d(0.5)
        self.dropout4 = nn.Dropout2d(0.5)


    def forward(self, x):
        x = F.relu(self.conv11(x))
        x = F.relu(self.conv12(x))
        x = F.max_pool2d(x, (2, 2))
        x = self.dropout1(x)

        x = F.relu(self.conv21(x))
        x = F.relu(self.conv22(x))
        x = F.max_pool2d(x, (2, 2))
        x = self.dropout2(x)

        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)
        x = self.fc2(x)
        x = self.dropout4(x)
        return self.fc3(x)

確認

In [ ]:
model = Cifar100Model().to(device)
print(model)

Cifar100Model(
  (conv11): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv21): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv22): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=100, bias=True)
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.25, inplace=False)
  (dropout3): Dropout2d(p=0.5, inplace=False)
  (dropout4): Dropout2d(p=0.5, inplace=False)
)


損失関数・オプティマイザー・スケジュール

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.9)

# **学習関数**

In [ ]:
global_step = 0

def train(epoch, writer):
    model.train()
    scheduler.step()

    print("\n--- Epoch : %2d ---" % epoch)
    print("lr : %f" % optimizer.param_groups[0]['lr'])

    steps = len(ds_train)//batch_size
    for step, (images, labels) in enumerate(dataloader_train, 1):
        global global_step
        global_step += 1

        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if step % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch, epochs, step, steps, loss.item()))
            writer.add_scalar('train/train_loss', loss.item() , global_step)


評価関数

In [ ]:
def eval(epoch, writer):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for (images, labels) in dataloader_test:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print("Val Acc : %.4f" % (correct/total))
    writer.add_scalar('eval/val_acc', correct*100/total, epoch)

追加


In [ ]:
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.0 MB/s 


# **学習実行**

In [ ]:
from tensorboardX import SummaryWriter
writer = SummaryWriter()

for epoch in range(1, epochs+1):
    train(epoch, writer)
    eval(epoch, writer)

writer.close()

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



--- Epoch :  1 ---
lr : 0.001000


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Epoch [1/150], Step [100/500], Loss: 4.3823
Epoch [1/150], Step [200/500], Loss: 4.1960
Epoch [1/150], Step [300/500], Loss: 4.1547
Epoch [1/150], Step [400/500], Loss: 4.0561
Epoch [1/150], Step [500/500], Loss: 4.1552
Val Acc : 0.1200

--- Epoch :  2 ---
lr : 0.001000
Epoch [2/150], Step [100/500], Loss: 3.7239
Epoch [2/150], Step [200/500], Loss: 3.8801
Epoch [2/150], Step [300/500], Loss: 3.5869
Epoch [2/150], Step [400/500], Loss: 3.4563
Epoch [2/150], Step [500/500], Loss: 3.5150
Val Acc : 0.1927

--- Epoch :  3 ---
lr : 0.001000
Epoch [3/150], Step [100/500], Loss: 3.5483
Epoch [3/150], Step [200/500], Loss: 3.4712
Epoch [3/150], Step [300/500], Loss: 3.5441
Epoch [3/150], Step [400/500], Loss: 3.3737
Epoch [3/150], Step [500/500], Loss: 3.3220
Val Acc : 0.2467

--- Epoch :  4 ---
lr : 0.001000
Epoch [4/150], Step [100/500], Loss: 3.5068
Epoch [4/150], Step [200/500], Loss: 3.1937
Epoch [4/150], Step [300/500], Loss: 3.1034
Epoch [4/150], Step [400/500], Loss: 3.2749
Epoch [4/15

# **保存**

In [ ]:
torch.save(model.state_dict(), 'model_cifar100.pkl')

In [ ]:
import torch

# state_dict()の表示
for key in model.state_dict():
    print(key, ": ", model.state_dict()[key].size())

# 保存
torch.save(model.state_dict(), "model_cnn100.pth")  

conv11.weight :  torch.Size([32, 3, 3, 3])
conv11.bias :  torch.Size([32])
conv12.weight :  torch.Size([32, 32, 3, 3])
conv12.bias :  torch.Size([32])
conv21.weight :  torch.Size([64, 32, 3, 3])
conv21.bias :  torch.Size([64])
conv22.weight :  torch.Size([64, 64, 3, 3])
conv22.bias :  torch.Size([64])
fc1.weight :  torch.Size([1024, 4096])
fc1.bias :  torch.Size([1024])
fc2.weight :  torch.Size([512, 1024])
fc2.bias :  torch.Size([512])
fc3.weight :  torch.Size([100, 512])
fc3.bias :  torch.Size([100])
